In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

In [2]:
def install_tidp(line, block_mvt=29e-3):
    tidp_ap_tot = 147e-3
    line.discard_tracker()
    tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
    line.collimators.install(names=['tidp.11434'], elements=[tidp])
    return tidp

def install_tcsm(line):
    tcsm = xc.EverestCollimator(length=1.83, gap=5, material=xc.materials.Carbon) # length is 1.83
    line.collimators.install(names=['tcsm.51932'], elements=[tcsm])
    return tcsm

def install_offmom_bpms_colls(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    tw = line.twiss()
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    colls = []
    aper_to_remove = []
    for nn in offmom_bpms:
        aper_to_remove.append(f'{nn}.a_aper')
        aper_to_remove.append(f'{nn}.b_aper')
        if line[nn+'.a_aper'].__class__.__name__ == 'LimitEllipse':
            jaw = line[nn+'.a_aper'].a
        else:
            jaw = line[nn+'.a_aper'].max_x
        
        colls.append(xc.EverestCollimator(length=line[nn].length, material=xc.materials.Beryllium, jaw=jaw))
    line.remove(aper_to_remove)
    line.collimators.install(names=offmom_bpms, elements=colls)
    return colls

def remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    "Remove apertures of off-momentum BPMs which give flanges as bottlenecks"
    tw = line.twiss()
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    aper_to_remove = [f'{name}{suffix}' for name in offmom_bpms for suffix in ('.a_aper', '.b_aper')]
    line.remove(aper_to_remove)

In [3]:
L1 = 34e-3
L2 = 70e-3

r1 = 41.5e-3
r2 = 60e-3-1.5e-3
r3 = 51.5e-3

changed = [10110, 11110, 11310, 12510, 13510, 20910, 21110, 22510, 23510, 30110, 30910,
32510, 33510, 40110, 40910, 41110, 42510, 42710, 51110, 52510, 53510, 60110,
61110, 62510, 63510]

In [4]:
def find_pipe_edges(line, pipe_data):
    tt = line.get_table()

    #pipe names
    pipe_names = list(pipe_data.keys())
    apers_to_add = {}
    
    for pipe_name in pipe_names:
        if 'edge_names' in pipe_data[pipe_name]:
            assert pipe_data[pipe_name]['edge_names'][0] in tt.name and pipe_data[pipe_name]['edge_names'][1] in tt.name, f"Pipe edge names {pipe_data[pipe_name]['edge_names']} not found in line for {pipe_name}"
        
        else:
            assert 'edge_positions' in pipe_data[pipe_name], f"Please provide edge names or edge positions for {pipe_name}"

            #Looking if apertures already exist in line
            pipe_start = None
            pipe_end = None
            for nn in tt.rows[(tt.s > pipe_data[pipe_name]['edge_positions'][0] - 1e-3 ) & (tt.s < pipe_data[pipe_name]['edge_positions'][0] + 1e-2 )].name:
                if line[nn].__class__.__name__.startswith('Limit'):
                    pipe_start = nn

            for nn in tt.rows[(tt.s > pipe_data[pipe_name]['edge_positions'][-1] - 1e-3 ) & (tt.s < pipe_data[pipe_name]['edge_positions'][-1] + 1e-2 )].name:
                if line[nn].__class__.__name__.startswith('Limit'):
                    pipe_end = nn
                    break
            
            if pipe_start is None or pipe_end is None:
                apers_to_add[pipe_name] = {'positions': pipe_data[pipe_name]['edge_positions'],
                                           'names': [f'{pipe_name}.a_aper', f'{pipe_name}.b_aper']}
                pipe_data[pipe_name]['edge_names'] = [f'{pipe_name}.a_aper', f'{pipe_name}.b_aper']
            else:
                pipe_data[pipe_name]['edge_names'] = [pipe_start, pipe_end]

    if len(list(apers_to_add.keys()))>0:
        insertions = []
        for pipe_name in apers_to_add.keys():
            line.env.elements[apers_to_add[pipe_name]['names'][0]] = pipe_data[pipe_name]['apertures'][0].copy()
            line.env.elements[apers_to_add[pipe_name]['names'][1]] = pipe_data[pipe_name]['apertures'][-1].copy()

        insertions += [line.env.place(apers_to_add[pipe_name]['names'][0], at=apers_to_add[pipe_name]['positions'][0]),
                        line.env.place(apers_to_add[pipe_name]['names'][1], at=apers_to_add[pipe_name]['positions'][1])]

        line.insert(insertions, s_tol=1e-6)

    return pipe_data

In [ ]:
def get_slicing_strategies(line, pipe_data):
    slicing_strategies = [xt.Strategy(slicing=None)]
    pipe_names = list(pipe_data.keys())
    tt = line.get_table()

    active_elements = []
    # Finding active elements in the pipe
    for pipe_name in pipe_names:
        pipe_start, pipe_end = pipe_data[pipe_name]['edge_names']
        for nn, ee in zip(tt.rows[f'{pipe_start}>>1':pipe_end].name, tt.rows[f'{pipe_start}>>1':pipe_end].element_type):
            if ee.startswith('Drift') or ee.startswith('Marker') or ee.startswith('Limit'):
                continue
            else:
                active_elements.append(nn)
    
    for nn in active_elements:
        slicing_strategies.append(xt.Strategy(slicing=xt.Teapot(5), name=nn))
    return slicing_strategies

In [ ]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
tt1 = line.get_table()
remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)
tt = line.get_table()
# tw = line.twiss()

#shift VEB apertures by 5.3 mm to account for the sagitta
veb_b_apers = tt.rows['veb.*.b_aper'].name
for name in veb_b_apers:
    line[name].shift_x += 5.3e-3

veb_a_apers = tt.rows['veb.*.a_aper'].name
for name in veb_a_apers:
    line[name].shift_x += 5.3e-3
#Houdt de a aperture steek?
tt = line.get_table()
tw = line.twiss()


pipe_data = {}

for nn in changed:
    flange_number = nn-9
    pipe_name = f'vcak.{flange_number}'
    
    L3 = line.get_s_position(f'vcak.{flange_number}.b_aper') - line.get_s_position(f'vcak.{flange_number}.a_aper') - L1 - L2
    if L3 < 0:
        raise ValueError(f'Pipe {pipe_name} too short to fit all apertures, please adjust L1 and L2')
    
    pipe_data[pipe_name] = {'apertures': [xt.LimitEllipse(a=r1, b=r1, shift_x=-5.3e-3), 
                                  xt.LimitEllipse(a=r2, b=r2, shift_x=-5.3e-3),
                                  xt.LimitEllipse(a=r3, b=r3)],
                            'lengths': [L1, L2, L3],
                            #'edge_names': [f'{pipe_name}.a_aper', f'{pipe_name}.b_aper'],
                            'edge_positions': [line.get_s_position(f'vcak.{flange_number}.a_aper'), line.get_s_position(f'vcak.{flange_number}.b_aper')]
                            }
    

pipe_data = find_pipe_edges(line, pipe_data)

slicing_strategies = get_slicing_strategies(line, pipe_data)
line.slice_thick_elements(slicing_strategies)

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
The line already has an associated tracker


Slicing line:   0%|          | 0/32246 [00:00<?, ?it/s]

{'sps$start': ['sps$start'],
 'begi.10010': ['begi.10010'],
 'qf.10010': ['qf.10010'],
 'drift_0..0': ['drift_0..0'],
 'veqf.10010.b_aper': ['veqf.10010.b_aper'],
 'drift_0..1': ['drift_0..1'],
 'veba.10030.a_aper': ['veba.10030.a_aper'],
 'drift_0..2': ['drift_0..2'],
 'mba.10030_entry': ['mba.10030_entry'],
 'mba.10030..entry_map': ['mba.10030..entry_map'],
 'mba.10030..0_aper2': ['mba.10030..0_aper2'],
 'mba.10030..0': ['mba.10030..0'],
 'mba.10030..1_aper2': ['mba.10030..1_aper2'],
 'mba.10030..1': ['mba.10030..1'],
 'mba.10030..2_aper2': ['mba.10030..2_aper2'],
 'mba.10030..2': ['mba.10030..2'],
 'mba.10030..3_aper2': ['mba.10030..3_aper2'],
 'mba.10030..3': ['mba.10030..3'],
 'mba.10030..4_aper2': ['mba.10030..4_aper2'],
 'mba.10030..4': ['mba.10030..4'],
 'mba.10030..5_aper2': ['mba.10030..5_aper2'],
 'mba.10030..5': ['mba.10030..5'],
 'mba.10030..6_aper2': ['mba.10030..6_aper2'],
 'mba.10030..6': ['mba.10030..6'],
 'mba.10030..7_aper2': ['mba.10030..7_aper2'],
 'mba.10030..7': 

In [36]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
tt1 = line.get_table()
remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)
tt = line.get_table()
# tw = line.twiss()

#shift VEB apertures by 5.3 mm to account for the sagitta
veb_b_apers = tt.rows['veb.*.b_aper'].name
for name in veb_b_apers:
    line[name].shift_x += 5.3e-3

veb_a_apers = tt.rows['veb.*.a_aper'].name
for name in veb_a_apers:
    line[name].shift_x += 5.3e-3
#Houdt de a aperture steek?
tt = line.get_table()
tw = line.twiss()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [6]:
tt = line.get_table()

In [8]:
tt.rows['mbb.12.*']

Table: 500 rows, 11 cols
name                             s element_type        isthick isreplica parent_name iscollective ...
mbb.12070_entry            656.709 Marker                False     False None               False
mbb.12070..entry_map       656.709 ThinSliceRBendEntry   False     False mbb.12070          False
mbb.12070..0_aper2         656.709 LimitRect             False     False None               False
mbb.12070..0               656.709 ThickSliceRBend        True     False mbb.12070          False
mbb.12070..1_aper2         657.335 LimitRect             False     False None               False
mbb.12070..1               657.335 ThickSliceRBend        True     False mbb.12070          False
mbb.12070..2_aper2         657.961 LimitRect             False     False None               False
mbb.12070..2               657.961 ThickSliceRBend        True     False mbb.12070          False
mbb.12070..3_aper2         658.587 LimitRect             False     False None            

In [12]:
tt.rows['qd.12510<<40':'qd.12510>>1'].show()

name                              s element_type       isthick isreplica parent_name        ...
mbb.12490..4                793.844 ThickSliceRBend       True     False mbb.12490         
mbb.12490..5_aper2           794.47 LimitRect            False     False None              
mbb.12490..5                 794.47 ThickSliceRBend       True     False mbb.12490         
mbb.12490..6_aper2          795.096 LimitRect            False     False None              
mbb.12490..6                795.096 ThickSliceRBend       True     False mbb.12490         
mbb.12490..7_aper2          795.722 LimitRect            False     False None              
mbb.12490..7                795.722 ThickSliceRBend       True     False mbb.12490         
mbb.12490..8_aper2          796.348 LimitRect            False     False None              
mbb.12490..8                796.348 ThickSliceRBend       True     False mbb.12490         
mbb.12490..9_aper2          796.974 LimitRect            False     False Non

In [13]:
tt.rows['vcak.13501.a_aper<<10':'vcak.13501.a_aper>>50'].show()

name                              s element_type        isthick isreplica parent_name        ...
mbb.13490..8_aper2          1116.32 LimitRect             False     False None              
mbb.13490..8                1116.32 ThickSliceRBend        True     False mbb.13490         
mbb.13490..9_aper2          1116.95 LimitRect             False     False None              
mbb.13490..9                1116.95 ThickSliceRBend        True     False mbb.13490         
mbb.13490..10_aper2         1117.58 LimitRect             False     False None              
mbb.13490..exit_map         1117.58 ThinSliceRBendExit    False     False mbb.13490         
mbb.13490_exit              1117.58 Marker                False     False None              
drift_299..0                1117.58 DriftSlice             True     False drift_299         
vebb.13490.b_aper           1117.77 LimitRect             False     False None              
drift_299..1                1117.77 DriftSlice             True   

# Remake more general

1) Find active elements in between MBB and QD last aperture
2) Slice thick elements 
3) Define collimator lengths
4) Remove last MBB aperture and VEBB.B aperture and other apertures in the way

## 1. Find active elements in between MBB and QD

Aperture or active element marks collimator lengths

In [28]:
start = 'vebb.13490.b_aper'
end = 'veqd.13510.b_aper'

tt = line.get_table()
names, elements = tt.rows[start:end].name, tt.rows[start:end].element_type

In [30]:
tt.rows[start:end]

Table: 29 rows, 11 cols
name                              s element_type isthick isreplica parent_name        iscollective ...
vebb.13490.b_aper           1117.77 LimitRect      False     False None                      False
drift_299..1                1117.77 DriftSlice      True     False drift_299                 False
vcak.13501.a_aper           1117.77 LimitEllipse   False     False None                      False
drift_299..2                1117.77 DriftSlice      True     False drift_299                 False
lod.13502                   1117.88 Octupole        True     False None                      False
drift_300..0                1118.55 DriftSlice      True     False drift_300                 False
vcak.13501.b_aper           1118.63 LimitEllipse   False     False None                      False
drift_300..1                1118.63 DriftSlice      True     False drift_300                 False
vtto.13505.a_aper           1118.63 LimitEllipse   False     False None          

In [25]:
tt.rows[start:end]

Table: 0 rows, 11 cols

In [37]:
elements_to_slice = []
collimators = {}
qd_number = 13510

material = xc.materials.Beryllium
X0_air = 301
air = xc.Material(radiation_length=X0_air, name="Air (1 atm 20C)")
material = air


coll_length = 0
start_ele = start
insertions = []
for nn, ee in zip(names, elements):
    if ee.startswith('Drift') or ee.startswith('Marker'):
        if hasattr(line[nn], 'length'):
            coll_length += line.env.elements[nn].length
        elif hasattr(line[nn], '_parent'):
            coll_length += line.env.elements[nn]._parent.length * line.env.elements[nn].weight
        
    elif ee.startswith('Limit'):
        if coll_length > 0:
            if start_ele == start:
                collimators[f'vcak.{qd_number-9}'] = {
                    'coll_elem': xc.EverestCollimator(length=coll_length, material=air, jaw=r2),
                    'ref_ele': start_ele
                }
                line.env.elements[f'vcak.{qd_number-9}.coll'] = collimators[f'vcak.{qd_number-9}']['coll_elem']
                insertions.append(line.env.place(f'vcak.{qd_number-9}.coll',at=f'{collimators[f'vcak.{qd_number-9}']['ref_ele']}@end', anchor='start'))
            
                ref_ele = nn
                coll_length = 0
    
    else:
        if coll_length > 0:
            collimators[f'{start_ele}.coll'] = {
                'coll_elem': xc.EverestCollimator(length=coll_length, material=air, jaw=r3),
                'ref_ele': start_ele
            }
            line.env.elements[f'{start_ele}.coll'] = collimators[f'{start_ele}.coll']['coll_elem']
            insertions.append(line.env.place(f'{start_ele}.coll',at=f'{collimators[f'{start_ele}.coll']['ref_ele']}@end', anchor='start'))
            ref_ele = nn
            coll_length = 0
        start_ele = nn
        elements_to_slice.append(nn)



In [38]:
line.insert(insertions, s_tol=1e-6)

Slicing line:   0%|          | 0/32246 [00:00<?, ?it/s]

AssertionError: Negative drift length: -0.0009000000000014552, upstream of vebb.13490.b_aper.coll

In [39]:
tt = line.get_table()

In [40]:
tt.rows[start:end]

Table: 30 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
vebb.13490.b_aper       1117.77 LimitRect      False     False None               False
drift_299..1            1117.77 DriftSlice      True     False drift_299          False
vcak.13501.a_aper       1117.77 LimitEllipse   False     False None               False
drift_299..2..0         1117.77 DriftSlice      True     False drift_299          False
drift_299..2..1         1117.87 DriftSlice      True     False drift_299          False
lod.13502               1117.88 Octupole        True     False None               False
drift_300..0            1118.55 DriftSlice      True     False drift_300          False
vcak.13501.b_aper       1118.63 LimitEllipse   False     False None               False
drift_300..1            1118.63 DriftSlice      True     False drift_300          False
vtto.13505.a_aper       1118.63 LimitEllipse   False     False None               False
...


In [41]:
insertions

[Place(vcak.13501.coll, at=0, from_=vebb.13490.b_aper, anchor=start, from_anchor=end),
 Place(vebb.13490.b_aper.coll, at=0, from_=vebb.13490.b_aper, anchor=start, from_anchor=end),
 Place(lod.13502.coll, at=0, from_=lod.13502, anchor=start, from_anchor=end),
 Place(lsd.13505.coll, at=0, from_=lsd.13505, anchor=start, from_anchor=end),
 Place(mdv.13507.coll, at=0, from_=mdv.13507, anchor=start, from_anchor=end)]